In [1]:
# If you what use ngrams.py you should use python2
# Or, otherwise, you need to modify ngrams.py by yourself in order to use it in python3.
from __future__ import print_function
from __future__ import unicode_literals

import re
import numpy as np
from ngrams import ngrams
from collections import defaultdict
from math import log, pow, inf

In [2]:
small_corpus = ['Why dont we start here',
                  'Why dont we end there',
                  'Let us start with a few other examples',
                  'We never start with an example with so few tokens',
                  'Tokens can be words that we start with in example docs'
                  ]

# HW3: Language Modeling
For this part of the assignment, you will implement two simple count-based n-gram language models: one based on maximum-likelihood estimation, and another based on Witten-Bell smoothing. The data you will be using is a subset of the Penn Treebank's tagged Wall Street Journal articles on which we have done some initial processing. There are two versions of the data for this assignment:

##### wsj.pos.gz
##### wsj-normalized.pos.gz
The difference is that, in the second (normalized) version of the data, we have collapsed some entries from certain tag categories (e.g. CDs, NNPs, etc.) into type-tokens to help reduce sparsity. Take a look at the data and see for yourself. Consider: what would be the benefits and drawbacks to this method of sparsity reduction? Note that, for this part of the assignment, the tags are un-necessary, so you'll want to work with the un-normalized version of the corpus.

https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

### Task 1: produce a tag-free corpus

For this task, you have two jobs. 
* First, you need to write a function to filter out all tags. 
* Second, Make sure your code works for both wsj.pos.gz and wsj-normalized.pos.gz

####What to turn in
* your code
* some samples to show me that your code works as it should be

In [3]:
import gzip

#regex = re.compile(r"/[A-Z$,.]{1,4}") #stackoverflow says unnecessary, cached anyway
maxln = 7

def rem_tag(line):
    line = line.decode('utf-8')
    return re.sub(r"/[A-Z$,.`']{1,4}",'',line)

print("######################## WSJ.POS Examples########################")
with gzip.open('wsj.pos.gz','r') as wsj:
    count = 0
    for line in wsj:
        print(line)
        print("\n")
        line = rem_tag(line)
        print(line)
        count += 1
        if count > maxln:
            break

print("######################## WSJ.NORMALIZED Examples:########################")
with gzip.open('wsj-normalized.pos.gz','r') as wsj:
    count = 0
    for line in wsj:
        print(line)
        print("\n")
        line = rem_tag(line)
        print(line)
        count += 1
        if count > maxln:
            break


######################## WSJ.POS Examples########################
b'Digital/NNP Equipment/NNP Corp./NNP reported/VBD a/DT 32/CD %/NN decline/NN in/IN net/JJ income/NN on/IN a/DT modest/JJ revenue/NN gain/NN in/IN its/PRP$ fiscal/JJ first/JJ quarter/NN ,/, causing/VBG some/DT analysts/NNS to/TO predict/VB weaker/JJR results/NNS ahead/RB than/IN they/PRP had/VBD expected/VBN ./.\n'


Digital Equipment Corp. reported a 32 % decline in net income on a modest revenue gain in its fiscal first quarter , causing some analysts to predict weaker results ahead than they had expected .

b'Although/IN the/DT second-largest/JJ computer/NN maker/NN had/VBD prepared/VBN Wall/NNP Street/NNP for/IN a/DT poor/JJ quarter/NN ,/, analysts/NNS said/VBD they/PRP were/VBD troubled/VBN by/IN signs/NNS of/IN flat/JJ U.S./NNP orders/NNS and/CC a/DT slowdown/NN in/IN the/DT rate/NN of/IN gain/NN in/IN foreign/JJ orders/NNS ./. The/DT Maynard/NNP ,/, Mass./NNP ,/, company/NN is/VBZ in/IN a/DT transition/NN in/IN wh

<font color="red">Self assessment:</font>

In [4]:
#Constructing Corpus
wsj_corpus = []
wsj_norm_corpus = []
with gzip.open('wsj.pos.gz','r') as wsj:
    for line in wsj:
        line = rem_tag(line)
        wsj_corpus += line.replace(',','').split(' .')

with gzip.open('wsj-normalized.pos.gz','r') as wsj:
    for line in wsj:
        line = rem_tag(line)
        wsj_norm_corpus += line.replace(',','').split(' .')


In [5]:
print("WSJ:", wsj_corpus[5])
print("WSJ NORM: ",wsj_norm_corpus[5])


WSJ: Net for the quarter ended Sept. 30 fell to $ 150.8 million  or $ 1.20 a share  from $ 223 million  or $ 1.71 a share  a year ago
WSJ NORM:  Net for the quarter ended <NNP> <CD> fell to $ <CD> <CD>  or $ <CD> a share  from $ <CD> <CD>  or $ <CD> a share  a year ago


### Maximum Likelihood
Now, start by producing code to compute maximum-likelihood estimate probabilities. Your code should be configurable with respect to the n-gram order- i.e., you should be able to set it to compute bigram, trigram, 4-gram, etc. probabilities. Refer to J&M and the lecture slides for definitions as needed. If you would like to write your own n-gram tokenization code, feel free to do so, but you may also use the ngrams.py utility class which contains a routine to take a list of tokens and produce a stream of n-grams with appropriate padding for the start and end of sentences.

#### Tip: 
* Start with a very small "toy" corpus of just a couple of sentences for debugging. 

* As discussed in class, I strongly recommend using nested defaultdicts as the foundational data structure for your language model, where the "outer" key is the prefix, and the value retrieved by that prefix is a second defaultdict  containing possible suffices for that prefix, each of which is an "inner" key. E.g., p("TRUTHS" | "HOLD THESE") would be retrieved by first looking up "HOLD THESE" and then from the resulting dictionary, looking up "TRUTHS": prob = trigrams[("HOLD","THESE")]["TRUTHS"] . Note that this arrangement makes it very easy to e.g. find out the number of times a given history occurs, the total probability mass assigned to all of a history's continuations, etc., all of which will be extremely helpful in the next part of the assignment.

* Use tuples to represent prefixes. E.g., instead of the string "HOLD THESE", use the tuple ("HOLD", "THESE"). Note that, in Python, lists are mutable, and therefore may not be used as keys in dictionaries- but tuples are immutable, and so make excellent keys.

* Don't forget about numerical underflow issues! You'll want to represent probabilities as negative base-2 log probabilities, and modify your arithmetic accordingly. I recommend experimenting with [the bitweight Python library](https://github.com/stevenbedrick/bitweight) (see its unit tests for example usage).
* 

#### What to turn in:
* your code 
* use your code to create a simple language model for small_corpus named as small_lm and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create language model for wsj.pos.gz named as wsj_lm

In [155]:
def log2(x):
    """Return -log2(x)"""
    if x == 0:
        return inf
    return -log(x, 2)

def pow2(x):
    """Return 2^-x"""
    return pow(2,-x)

def make_dict(corpus, order):
    """Returns a nested defaultdict(defaultdict), formatted as follows:
        [prefix]:[suffix]:prefix/suffix count"""
    corpus_list = []
    corpus_dict = defaultdict(lambda: defaultdict(int))

    corpus_list += [corpus[i].lower().split() for i in range(len(corpus))] #corpus[i].lower().split() ?
    chunks = ngrams(corpus_list, order)
    lchunks = list(chunks)
    
    for i in range(len(lchunks)):
        corpus_dict[lchunks[i][0] ][lchunks[i][1]] += 1

    return corpus_dict

def mle_solo(corp_dict, entry):
    """single n-gram probability function
    Input: dictionary(dictionary()): corp_dict, String: sentence
    Returns: ngrams as a list
    Order identified from dictionary used"""

    if entry[1] not in corp_dict[entry[0]]:
        return 0
    total = 0
    for val in corp_dict[entry[0]]:
        total += corp_dict[entry[0]][val]

    #prob = corp_dict[entry[0]][entry[1]] / total
    prob = log2(corp_dict[entry[0]][entry[1]]) - log2(total)
    prob = pow2(prob)
    return prob

def mle100(corp_dict, sentence):
    """Multiply version of MLE, whole sentence (i.e. parsed through ngrams() for <S_0>, etc)"""
    acc = 1
    order = len(list(corp_dict.keys())[0]) + 1
    n = list(ngrams([sentence.split()], order))
    #print(n)
    for entry in n:
        if entry[1] not in corp_dict[entry[0]]:
            return 0
        total = 0
        for i in corp_dict[entry[0]]:
            total += corp_dict[entry[0]][i]
        #print(total)
        prob = corp_dict[entry[0]][entry[1]] / total
        acc = acc * prob
        #print("acc@", entry[0], ":", entry[1] ,acc)
    return acc

def mle_log(corp_dict, sentence):
    """Log version of MLE, returns in -log2"""
    sentence = sentence.lower()
    acc = 0 #in log2
    order = len(list(corp_dict.keys())[0]) + 1
    n = list(ngrams([sentence.split()], order))
    for entry in n:
        total = 0
        for i in corp_dict[entry[0]]:
            total += corp_dict[entry[0]][i]
        #print("total",total)
        if entry[1] not in corp_dict[entry[0]]:
            return inf
        suffix = corp_dict[entry[0]][entry[1]]
        if (suffix == 0):
            return inf
        prob = log2(suffix) - log2(total)
        #print("PROB", prob)
        assert prob >= 0, "-log2(prob) not > 0" #since negative log(fraction)
        acc = acc + prob
        #print("acc@", entry[0], ":", entry[1] ,acc)
    return acc

def print_dict(corpus_dict):
    keys = corpus_dict.keys()
    
    for key in sorted(keys):
        print(key)
        for key2 in sorted(corpus_dict[key].keys()):
            print("\t", key2, ":", corpus_dict[key][key2])

### Small Toy Test Case, BIGRAMS
small_lm = make_dict(small_corpus, 2)

### MLE with multiplication
print("=0 until smoothing:", mle100(small_lm, "start")) #0-probability case
print("mle100", mle100(small_lm, "we start here")) #non-zero probability case

### MLE with log
res1 = mle_log(small_lm, "start")
print("=0 until smoothing: log: ", res1, "decimal:", pow2(res1)) #0-probability case
res2 = mle_log(small_lm, "we start here") #non-zero probability case
print("mle log: ", res2, "decimal prob:", pow2(res2))

res3 = mle_log(small_lm, "Why dont we end there")
print("'why dont we end there':mle log: ", res3, "decimal prob:", pow2(res3))

small_lm1 = make_dict(small_corpus, 1)
testword = "us"
res = mle_log(small_lm1, testword)
print("unigram: ",testword, res, "decimal:", pow2(res))

=0 until smoothing: 0
mle100 0.025
=0 until smoothing: log:  inf decimal: 0.0
mle log:  5.321928094887362 decimal prob: 0.025000000000000012
'why dont we end there':mle log:  3.321928094887362 decimal prob: 0.1
unigram:  us 5.285402218862249 decimal: 0.025641025641025633


In [84]:
wsj_lm = make_dict(wsj_corpus, 2)
wsj_norm_lm = make_dict(wsj_norm_corpus, 2)

In [9]:
res = mle_log(wsj_lm, "In New York") #non-zero probability case
print("mle (negative) log: ", res, "decimal prob:", pow2(res))

res = mle_log(wsj_lm, "Pineapple") #zero probability case
print("mle (negative) log: ", res, "decimal prob:", pow2(res))

mle (negative) log:  16.0467925383373 decimal prob: 1.4771823564213143e-05
mle (negative) log:  inf decimal prob: 0.0


<font color="red">Self assessment:</font>

### Smoothing

Once you’ve got an unsmoothed model working, move on to implementing Witten-Bell smoothing. Refer to the slides and J&M for details on how that ought to work.

#### Tip: 
* You can modify an already-populated defaultdict to change its default value (for example, to store a default backoff value for a particular history) by changing the object’s default_factory attribute. Consult the documentation for examples of how this works.
* As defined, W-B smoothing is highly recursive; you may find it more efficient to re-frame the algorithm in iterative terms.
* As in the previous section, start small.
* [This may offer you some help on how to implement Witten-Bell smoothing](http://www.ee.columbia.edu/~stanchen/e6884/labs/lab3/x207.html)


#### What to turn in:
* your code 
* use your code to create a simple smoothed language model based on small_lm  and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create a smoothed language model based on wsj_lm

In [10]:
#Default value usage test>Pass
d1 = defaultdict(int)
d1["apple"] = 5
print(d1[5])
x = 6
d1.default_factory = lambda x=x: x #6
print(d1.default_factory(), d1[100], d1[5], d1["apple"]) #Result 6 6 0 5
#Note: d1[5] set to initial value 0 after first call
#Note: Use d1.default_factory() to call lambda function directly without creating new entry

d2 = defaultdict(lambda: defaultdict(int))
d2[1][2] = 15
x = 0.5
d2[1].default_factory = lambda x=x: x #0.5
print(d2[1][2], d2[1][111], d2[2][2]) #Expected 15, 0.5, 0

#Note: Changing lambda: 6 to lambda: x causes an issue.
#https://stackoverflow.com/questions/10452770/python-lambdas-binding-to-local-values
#https://stackoverflow.com/questions/21053988/lambda-function-acessing-outside-variable
print(d1.default_factory(), d1[100], d1[5], d1["apple"])
print(d2[1][2], d2[1][111], d2[2][2])

#Note: Use lambda x=x: x to capture

0
6 6 0 5
15 0.5 0
6 6 0 5
15 0.5 0


In [95]:
# n = 2
# models = [] #list of dict(dict(ints)); each entry is a 1-gram, 2-gram, ..., n-gram

############## Small Toy Test Case

#Initialize n-gram dictionaries from unigram to desired n-gram
# for i in range(n):
#     small_lm = make_dict(small_corpus, i+1)
#     models.append(small_lm.copy())
# #Verify
# #print_dict(models[n-1])

# #Build up lambda terms for unigram to (n)-gram, storing in models[]
# for i in range(0,n):
#     keys = models[i].keys()
#     print("KEYS: ", keys)
#     for key in keys:
#         inner_keys = models[i][key].keys()
#         print("INNER KEYS: ", inner_keys)
        
#         tokens = 0
#         for ikey in inner_keys:
#             tokens += models[i][key][ikey]

#         types = 0
#         types = len(inner_keys)
        
#         print("(tokens types)", tokens, types)
#         lamb = tokens / (tokens+types)
#         models[i][key].default_factory = lambda lamb=lamb: lamb

        #print("lambda for n =",(i+1),"key=",key,":", models[i][key].default_factory())

def make_lamb_dicts(models, n, corpus, debug = 0):
    #Initialize n-gram dictionaries from unigram to desired n-gram
    for i in range(n):
        lm = make_dict(corpus, i+1)
        models.append(lm.copy())
    #Verify
    #print_dict(models[n-1])

    #Build up lambda terms for unigram to (n)-gram, storing in models[]
    for i in range(0,n):
        keys = models[i].keys()
        if debug:
            print("KEYS: ", keys)
        for key in keys:
            inner_keys = models[i][key].keys()
            if debug:
                print("INNER KEYS: ", inner_keys)

            tokens = 0
            for ikey in inner_keys:
                tokens += models[i][key][ikey]

            types = 0
            types = len(inner_keys)
            
            if debug:
                print("(tokens types)", tokens, types)
            lamb = tokens / (tokens+types)
            models[i][key].default_factory = lambda lamb=lamb: lamb


In [145]:
n = 3 #up to trigrams created
testlambs = []
make_lamb_dicts(testlambs, n, small_corpus, debug = 0)
#print_dict(testlambs[1])

In [158]:
#Calculate Smoothed Probability, starting with unigram level (iterative)
#list_grams = list(ngrams([test_case.split()], 2))
#print(list_grams)

def wbSmooth(list_dicts, sentence, order, debug = 0):
    """
    Input:
        list_dicts: List of nested defaultdict(defaultdict(int)) structure containing n-gram counts
            from 1-gram to (order)-gram.
        sentence: Input sentence to calculate probability on
        order: degree of n-gram. Must be within provided (list_dict) range
    Output: 
    """
    sentence=sentence.lower()

    uni_tokens = sum(list_dicts[0][()].values())
    uni_types = len(list_dicts[0][()].keys())
    lamb_dot = uni_tokens / (uni_tokens+uni_types)
    unseen = 1/(uni_tokens+uni_types)
    if debug:
        print("uni_tokens, uni_types:", uni_tokens, uni_types)
        print("lambda.dot = ", lamb_dot)
        print("unseen", unseen)
    
    grams = list(ngrams([sentence.split()], order))
    print(order,"-grams:", grams)
    probs = []

#Calculate smoothed prob for every n-gram in (sentence) and add to accumulator
    for gram in grams:
        prefix = gram[0]
        suffix = gram[1]
        if debug:
            print("current gram: ", gram)

        ptemp =0.5

        for i in range(0,order):
            if i == 0: #base case, unigrams
                #p = lamb_dot * pow2(mle_log(list_dicts[i], suffix)) + unseen
                p = lamb_dot * (mle100(list_dicts[i], suffix)) + unseen
                #p = pow2( log2(lamb_dot) + log2((mle100(list_dicts[i], suffix))) ) + unseen #no effect
                ptemp = p
            else:
                gram_dict = list_dicts[i][prefix[order-i-1:]] #slice of prefix for current level of n-gram
                lamb = gram_dict.default_factory()
                #s = " ".join(prefix[order-i-1:])+" "+suffix #mle function takes [prefix, suffix]
                s = [prefix[order-i-1:], suffix]
                mle = mle_solo(list_dicts[i], s)
                #ptemp = lamb*mle + (1-lamb)*ptemp
                ptemp = pow2(log2(lamb)+log2(mle)) + pow2(log2((1-lamb))+log2(ptemp))
                
                if debug:
                    print("s: ",s, "ptemp: ", ptemp)
                    print("mle", s, mle_solo(list_dicts[i], s))
            
            #print("prefix", (gram[order-2-i],) ) #Unigram prefix
        probs.append( ptemp)
#Multiply (log form) individual gram probabilities together
    ret = 0
    for prob in probs:
        ret += log2(prob)
    return pow2(ret)

test_case = "let"
print("unigram smoothed: ",test_case, wbSmooth(testlambs ,test_case, 1))
test_case = "let us"
print("unigram smoothed: ",test_case, wbSmooth(testlambs ,test_case, 2, debug=1))
test_case = "why why"
print("bigram unseen pair smoothed: ",test_case, wbSmooth(testlambs ,test_case, 2, debug=0))
test_case = "us with"
print("bigram unseen pair smoothed: ",test_case, wbSmooth(testlambs ,test_case, 2, debug=0))
test_case = "chicken chicken"
print("bigram unseen tokens smoothed: ",test_case, wbSmooth(testlambs ,test_case, 2, debug=0))

testword = "let"
res = mle_log(small_lm1, testword)
print("unigram unsmoothed: ",testword, res, "decimal:", pow2(res))

#Bigger test cases
test_case = "why dont we end there"
print("bigram completely seen sentence: ",test_case, wbSmooth(testlambs ,test_case, 2, debug=0))
#Trigrams test for fun
test_case = "why dont we end there"
print("trigram completely seen sentence: ",test_case, wbSmooth(testlambs ,test_case, 3, debug=0))
test_case = "let us start here"
print("trigram sentence: ",test_case, wbSmooth(testlambs ,test_case, 3, debug=0))

#print_dict(testlambs[0])



1 -grams: [((), 'let')]
unigram smoothed:  let 0.03125
uni_tokens, uni_types: 39 25
lambda.dot =  0.609375
unseen 0.015625
2 -grams: [(('<S_0>',), 'let'), (('let',), 'us'), (('us',), '</S_0>')]
current gram:  (('<S_0>',), 'let')
s:  [('<S_0>',), 'let'] ptemp:  0.12500000000000003
mle [('<S_0>',), 'let'] 0.2
current gram:  (('let',), 'us')
s:  [('let',), 'us'] ptemp:  0.515625
mle [('let',), 'us'] 1.0
current gram:  (('us',), '</S_0>')
s:  [('us',), '</S_0>'] ptemp:  0.0078125
mle [('us',), '</S_0>'] 0
unigram smoothed:  let us 0.0005035400390625003
2 -grams: [(('<S_0>',), 'why'), (('why',), 'why'), (('why',), '</S_0>')]
bigram unseen pair smoothed:  why why 1.9779911747685202e-05
2 -grams: [(('<S_0>',), 'us'), (('us',), 'with'), (('with',), '</S_0>')]
bigram unseen pair smoothed:  us with 4.23855251736111e-06
2 -grams: [(('<S_0>',), 'chicken'), (('chicken',), 'chicken'), (('chicken',), '</S_0>')]
bigram unseen tokens smoothed:  chicken chicken 1.6954210069444448e-06
unigram unsmoothed:

### Verifying small corpus by hand:
lambda.dot = tokens/(tokens+types) = 39/(39+25) = 39/64

Unigram "let" occurs once in the corpus of 39 tokens

therefore, P.wb(let) = (39/64) * (1/39) + (1/64) = **2/64**, 


consistent with the result above `unigram smoothed:  let 0.03125`

The Bigram of `<S_0>` let, is then equivalent to (5/9) * (1/5) + (4/9) * (2/64) = **0.125**
with lambda of `<S_0>` = 5/9
MLE(let|`<S_0>` ) = 1/5

consistent with the result above `s:  [('<S_0>',), 'let'] ptemp:  0.125`

### Analysis
The following test cases and outputs taken from above:

The results are as expected. As we move from a sentence with more bigrams in the corpus (e.g. for "let us", both 'S_0 let' AND 'let us' occur in the corpus) to one with 0-bigrams, but still with unigrams in the corpus ("us with"), to 0 sightings in the corpus at all ("chicken chicken"), the probability goes down.

    Input            Probability         Links
    let us           5.035 e-04          2/3 bigrams exist, 2/2 unigrams exist
    why why          1.977 e-05          1/3 bigrams exist, 2/2 unigrams exist
    us with          4.23 e-06           0/3 bigrams exist, 2/2 unigrams exist
    chicken chicken  1.69 e-06           0/3 bigrams exist, 0/2 unigrams exist

<font color="red">Self assessment:</font>

### Evaluation via Perplexity
Explore the effects of n-gram order using perplexity. Perform ten-fold cross-validation on the WSJ corpus. On each iteration, this will give you a different 90/10 training/test split; train a smoothed language model on the 9 training sections, and compute the average per-token perplexity of the tenth section. The slides from the language modeling lecture give the equation for perplexity computation (as does J&M chapter 4); you'll need to modify the equation a bit, since we're using log-probabilities. 

Now, try this for unigram, bigram, trigram, and 4-gram models. 

#### What to turn in
* your cross-validation function. You are not suppose to use any cross-validation function from any module. You should implement it by yourself.
* your perplexity function
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj.pos.gz
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj-normalized.pos.gz.
* Answer following 2 questions: 
    * How does perplexity change as the model order size increases?
    * How does perplexity change as the data changed?

<font color="red">Self assessment:</font>